In [7]:
import pandas as pd
dat = pd.read_csv('contextual_first_batch_both_202.csv',encoding = "ISO-8859-1")

In [20]:
infomedia = pd.read_excel('Infomedia2.xlsx',encoding = "ISO-8859-1")

In [26]:
for _field in ['Article Id','Topic Name']:
    infomedia[_field.replace(" ","")]=infomedia[_field]

In [42]:
adat = pd.merge( infomedia, dat,  how='left', left_on=['Article Id','Topic Name'], right_on = ['ArticleId','TopicName'])

In [54]:
all_dat = infomedia.set_index(['ArticleId','TopicName']).join(dat.set_index(['ArticleId','TopicName']), how='left',lsuffix='_left', rsuffix='_right')

In [53]:
dat = dat.drop_duplicates(subset=['ArticleId','TopicName'])

In [62]:
len(all_dat)

68129

In [64]:
all_dat.to_csv("batch2merged_fin.csv")

In [59]:
all_dat.fillna(0.0, inplace=True)

In [34]:
all_dat=all_dat.dropna(subset=['Processed Terms_right'])

In [11]:
all_dat['Score_all'] = all_dat['Score_all'].fillna(0)

In [60]:
all_dat['Score_all'] = all_dat['Score_all'].fillna(0.0)
all_dat['Score_all']=all_dat['Score_all'].apply(lambda x: abs(x) < 1 and 0 or x)
import numpy as np
all_dat['Score_all']=all_dat['Score_all'].apply(lambda x: abs(x) > 10 and np.sign(x)*10 or x)

In [61]:
ans_ = all_dat

In [40]:
def positive(k, ans_=all_dat,tol=1e-7):
    return ans_[k]>tol

def negative(k,ans_=all_dat, tol=1e-7):
    return ans_[k]<-tol

def neutral(k,ans_=all_dat, tol=1e-7):
    return abs(ans_[k])<tol

In [41]:
def correct_count(ans_):
    count_ = 0
    for fun_ in [positive, negative, neutral]:
        count_+=sum(fun_('Score_all')&fun_('SentimentScore'))
    return count_

from itertools import permutations
def incorrect_count(ans_): 
    count_ = 0 
    for fun1,fun2 in permutations([positive, negative, neutral],2): 
        count_+=sum(fun1('Score_all')&fun2('SentimentScore')) 
    return count_

In [42]:
correct_count(all_dat)/float(correct_count(all_dat)+incorrect_count(all_dat))

0.49614392927851192

In [45]:
ans_ = all_dat[['Score_all','SentimentScore']].dropna(axis=1, how='any')

In [50]:
len(dat)

66639

In [44]:
len(infomedia)

85202

In [19]:
infomedia['SentimentScore']=infomedia['SentimentScore'].apply(lambda x: float(x.replace(",",".")))

In [8]:
ans_ = dat.set_index(['ArticleId','TopicName']).join(infomedia.set_index(['ArticleId','TopicName']))

In [10]:
ans_.to_csv('batch2merged.csv')

In [159]:
ans_['Score_all']=ans_['Score_all'].fillna("NA")

In [160]:
ans_['Score_first']=ans_['Score_first'].fillna("NA")

## Infomedia positive, we positive

In [ ]:
sum((ans_['Score_all']>1e-7)&(ans_['SentimentScore']>1e-7))

## Infomedia negative, we negative

In [133]:
sum((ans_['Score_all']<-1e-7)&(ans_['SentimentScore']<-1e-7))

3971

## Infomedia positive, we neutral

In [134]:
sum((abs(ans_['Score_all'])<1e-7)&(ans_['SentimentScore']>1e-7))

10659

## Infomedia positive, we negative

In [135]:
sum((ans_['Score_all']<-1e-7)&(ans_['SentimentScore']>1e-7))

4537

## Infomedia neutral, we neutral

In [136]:
sum((abs(ans_['Score_all'])<0.3)&(abs(ans_['SentimentScore'])<1e-7))

10260

## Infomedia neutral, we positive

In [137]:
sum((ans_['Score_all']>1e-7)&(abs(ans_['SentimentScore'])<1e-7))

8652

## Infomedia neutral, we negative

In [138]:
sum((ans_['Score_all']<-1e-7)&(abs(ans_['SentimentScore'])<1e-7))

4691

## Infomedia negative, we neutral

In [139]:
sum((abs(ans_['Score_all'])<1e-7)&(ans_['SentimentScore']<-1e-7))

2208

## Infomedia negative, we positive

In [140]:
sum((ans_['Score_all']<-1e-7)&(ans_['SentimentScore']<-1e-7))

3971

In [149]:
def correct_count(ans_):
    return sum((abs(ans_['Score_all'])<1)&(abs(ans_['SentimentScore'])<1e-7))+sum((ans_['Score_all']>1e-7)&(ans_['SentimentScore']>1e-7))+sum((ans_['Score_all']>1e-7)&(ans_['SentimentScore']>1e-7))

In [ ]:
def incorrect_count(ans_):
    return sum((abs(ans_['Score_all'])<1e-7)&(abs(ans_['SentimentScore'])<1e-7))+sum((ans_['Score_all']>1e-7)&(ans_['SentimentScore']>1e-7))+sum((ans_['Score_all']>1e-7)&(ans_['SentimentScore']>1e-7))

In [143]:
def incorrect_count(ans_):
    return {
        'neutrals':sum((ans_['Score_all']<-1e-7)&(abs(ans_['SentimentScore'])<1e-7))+sum((ans_['Score_all']>1e-7)&(abs(ans_['SentimentScore'])<1e-7)),
        'positives':sum((abs(ans_['Score_all'])<1e-7)&(ans_['SentimentScore']>1e-7))+sum((ans_['Score_all']<-1e-7)&(ans_['SentimentScore']>1e-7)),
        'negatives':sum((ans_['Score_all']<-1e-7)&(ans_['SentimentScore']<-1e-7))+sum((abs(ans_['Score_all'])<1e-7)&(ans_['SentimentScore']<-1e-7))
    }

In [147]:
len(ans_)

73714

In [153]:
correct_count(ans_)

49764

In [154]:
49764/73714.0

0.6750956399055811

In [144]:
incorrect_count(ans_)

{'negatives': 6179, 'neutrals': 13343, 'positives': 15196}

In [146]:
correct_count(ans_)/float(sum(incorrect_count(ans_).values())+correct_count(ans_))

0.55167871900826448

In [92]:
(abs(neutrals[neutrals['SentimentScore']==0.0].Score_all)< 1e-7).mean()

0.51157994923857864